In [113]:
import os
import json
from IPython.display import HTML, display
try:
    (notebook_dir, e)
except:
    e = "a --- b"
    notebook_dir = "."

In [114]:
padding = {
    "s_": -80,
    "ks_": -180,
    "ts_": -380,
    "k_high": -280,
    "k_low": -390,
    "ta_": -250,
    "to_": -250,
    "tb_": -250,
}

### Nommage des fichiers : 
* Lettre concernée _ left ou right ou lettertoconnect_right ; (lettre concernée = celle qui est sélectionnée lors du surlignage)
* t_before_a_right pour les clusters conditionnés => "ta_" suffit si la lettre attendue est \_a; et ta_after_... pour l'autre sens
* ks_above_t
* **hz**_440 (ligne horizontale de largeur 440)
* vt_80_220 (vertical debut et fin) ; vt_**n**160_220 pour du négatif
* uL et uR (Left; Right par rapp à la direction vers laquelle l'arche pointe); bL et bR (direction de la pointe pour les heads) ; les autres lesttres sont en a_left; a_right; etc

### TODO :
* Faire un c_ (quart supérieur gauche) et un _c
* Faire toutes les formes de voyelles connectées avec F (a, o, p_right) ça sera plus simple.
* Pas besoin de faire un uL spécial, la lettre a un comportement totalement erratique, nommer spécialement ses deux formes et définir des connecteurs pour chacune
* Ne reste que les ligatures spéciales pour les voyelles précédent les clusters, les formes T basses, et "i" "l" "h" 

In [115]:
fea = '''
feature ccmp {
    script DFLT;
    language dflt required;
    script latn;
    language dflt required;

    # ======================================================== 1 CHOIX

    lookup ETAPE_1_CHOIX {  # Choix forme contextuelle de base

        # Lettre F

        sub [a p] f' by f_high;
        sub [o e] f' by f_low;

        sub f' [a c u] by f_high;
        sub k' [a c u b] by k_high;

        sub f' [o b] by f_low;  # This isnt optimal but it's easier to read
        sub k' [o b] by k_low;

        # Lettre B

        sub [o n f_low] b' by bR;
        sub b' by bL;

        # Lettre U

        sub [n] u' by uL;  # Ignore next letter
        sub u' [a e p o b c y i n] by uR;
        sub u' by uL;
        # Define all other forms of U as codas!

    } ETAPE_1_CHOIX;

    # ======================================================== 2 COMPOSANTS
    
    lookup ETAPE_2_COMPOSANTS {  # Decoupe tous les caracteres en composants

        # A E P | O Y

        sub a by a_ _a;
        sub o by o_ _o;
        sub e by a_ _e;
        sub p by a_ _p;
        sub y by o_ _y;
        sub c by c_ _c;

        # B
        
        sub bR by o_ _b;
        sub bL by b_ _o;

        # C

        # S

        sub s by s_ _s;

    } ETAPE_2_COMPOSANTS;

    # ======================================================== 3 FUSION
    
    lookup ETAPE_3_FUSION {  # Fusionne les composants des que possible

        # AO OA

        sub _a o_ by _ao_;
        sub _o a_ by _oa_;

        sub _oa_ a_ by _oa_;
        sub _ao_ o_ by _ao_;

        # S

        sub _s a_ by _sa_;
        sub _s b_ by _sb_;
        sub _s c_ by _sc_;

        # K

        sub k s_ by ks_;

        # T

        sub t s_ by ts_;
        sub t a_ by ta_;
        sub t o_ by to_;
        sub t b_ by tb_;
        sub t c_ by tc;

    } ETAPE_3_FUSION;

    # ======================================================== 4 ACCENTS
    
    # lookup ETAPE_4_ACCENTS {  # Modifie et positionne les accents
    # } ETAPE_4_ACCENTS;

    # ======================================================== 5 ESTHETIQUE

    lookup ETAPE_5_ESTHETIQUE {  # Applique les formes purement esthétiques
        
        # HORIZONTAL CONNECTORS

        sub [n f_low] a_' by hz_320_low a_;
        sub _a' [n f_low] by _a hz_320_low;
        sub _p' [n f_low] by _p hz_320_low;

        sub [n f_low] c' [n f_low] by hz_200_low c hz_200_low;  # c
        sub c' [n f_low] by c hz_200_low;
        sub [n f_low] c' by hz_200_low c;

        sub [n f_high] o_' by hz_320_high o_;  # afo marker
        sub _o' [n f_high] by _o hz_320_high;
        
        sub _e' [n f_low s] by hz_275_low;
        sub _b' [n f_low s] by hz_275_high;

        # U FORMS

        sub k_high uR' [n f_low] by hz_320_low u_after_high;
        sub k_high uR' by u_after_high;
        sub [n f_low] uL' by hz_320_low uL_after_hz_high;
        sub [k_high f_high] uL' by uL_after_hz_high;
        sub uR' n by uR_before_hz_high hz_320_low;
        sub uR' f_high by uR_before_hz_high;

        # ENDING FORMS

        sub [_a n f_high] s_' _s' by s_after_high;
        sub [f_low hz_275_low] _s' by s_after_low;
        sub _o' s_' by _os_;
        sub _os_ _s' by s_after_low;

    } ETAPE_5_ESTHETIQUE;

} ccmp;
'''

In [116]:
fea += '''
feature kern {
    script DFLT;
    language dflt;
    script latn;
    language dflt;

    # SUIVANT VOYELLE -80

    pos [_a _o] [a_ o_] -80;
    pos [_s] [o_] -80;
    pos [_a _o n f_low f_high _os_ hz_275_low] [s_after_high s_after_low] -80;

    # HZ LINES
    pos [_a _o _p] [hz_320_low hz_320_high] -320;  # N A DROITE
    pos [_a _o _p] [f_high f_low n] -70;
    
    pos [hz_320_low hz_320_high] [a_ o_ uL_after_hz_high] -320;  # N A GAUCHE
    pos [f_high f_low n] [a_ o_] -70;

    pos uL_after_hz_high hz_320_low -250;
    pos uR_before_hz_high hz_320_low -260;

    pos c hz_200_low -200;
    pos hz_200_low c -200;

    pos [hz_320_low hz_320_high hz_200_low uL_after_hz_high] [f_high f_low n] -70;
    pos [f_high f_low n] [hz_320_low hz_320_high hz_200_low uL_after_hz_high] -70;

    pos k_high c -40;

    # VOYELLES DEUX A DEUX

    # COMPOSANTS S

    # ELEMENTS HORIZONTAUX ESTHETIQUES
    
} kern;
'''

In [117]:
try: counter += 1
except: counter = 0
if not os.getcwd().endswith('bin'): 
    notebook_dir = os.getcwd()  # Actual working directory
    %cd ".\spetekkimyo\ffpython\bin\"
json.dump(padding, open(f'{notebook_dir}\\spetekkimyo\\input\\padding.json', 'w'))
with open(f'{notebook_dir}\\spetekkimyo\\input\\features.fea', 'w') as f: f.write(fea)
!"ffpython.exe" "..\..\generate.py" "{notebook_dir}\output\test.otf"

Imported 66 glyphs: a a_ b bL bR b_ c c_ e f f_high f_low hz_200_low hz_275_high hz_275_low hz_320_high hz_320_low hz_440_low i k ks ks_ k_high k_low n o o_ p s s_ s_after_high s_after_low t ta_ tb_ tc tks_ tk_high tk_low to_ ts_ u uL uL_after_hz_high uR uR_before_hz_high u_after_high y _a _an_ _ao_ _b _c _e _fc _na_ _nc_ _o _oa_ _os_ _p _s _sa_ _sb_ _sc_ _y
Imported features
Font generated at g:\GitHub\Spetekkimyo\output\test.otf


In [118]:
e = """
<span class="ss">tbfbsbnofas c</span> pb
"""

In [119]:
html = """<style>p {font-size: 100px;} @font-face {src:url('./output/test.otf?version=%s');font-family:'test';} .ss {font-family:'test';}</style>
<p>%s</p>""" % (counter, e)
display(HTML(html))

### Guidelines

Epaisseur du trait : 80

Largeur : **550** (275) -- consonants 320 - 440 (max d'épaisseur) - 550 (pointe droite)

Hauteur standard : 320 -- max = 730 -- low = -240

Départ : h=-12